## Collect and aggregate cable news data and Google Trends data

### Load R packages

In [1]:
library(hrbrthemes)
library(tidyverse)
library(lubridate)
library(newsflash)
library(magrittr)
library(stringr)
library(gtrendsR)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date


Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



### Create list of prominent people accused of sexual misconduct

In [2]:
alleg_list <- c(
    "Harvey Weinstein",
    "Chris Savino",
    "Robert Scoble",
    "Lockhart Steele",
    "Josh Besh",
    "Terry Richardson",
    "Leon Wieseltier",
    "Knight Landesman",
    "Rick Najera",
    "Mark Halperin",
    "Ken Baker",
    "Kevin Spacey",
    "Hamilton Fish",
    "Michael Oreskes",
    "Andy Dick",
    "Kirt Webster",
    "Brett Ratner",
    "Jeff Hoover",
    "David Guillod",
    "Benjamin Genocchio",
    "Louis CK",
    "Roy Moore",
    "Al Franken",
    "Richard Dreyfuss",
    "Gary Goddard",
    "Dustin Hoffman",
    "Robert Knepper",
    "Andrew Kreisberg",
    "Jeremy Piven",
    "Steven Seagal",
    "Tom Sizemore",
    "Jeffrey Tambor",
    "George Takei",
    "James Toback",
    "Matthew Weiner",
    "Stephen Blackwell",
    "Jann Wenner",
    "Matt Zimmerman",
    "Stephen Bittel",
    "Jeff Clemens",
    "Jack Latvala",
    "Michael Fallon",
    "Alex Gilady",
    "Danny Jordaan",
    "Sepp Blatter",
    "Glenn Thrush",
    "Charlie Rose",
    "John Lasseter",
    "Matt Lauer",
    "Garrison Keillor",
    "John Conyers",
    "Israel Horovitz",
    "Russell Simmons",
    "Nick Carter",
    "James Levine",
    "Ruben Kihuen",
    "Blake Farenthold",
    "Lorin Stein",
    "Danny Masterson",
    "Matt Dababneh",
    "David Sweeney",
    "Trent Franks",
    "Wes Goodman",
    "Tony Cornish",
    "Warren Moon"
  ) %>%
  str_to_lower() %>%
  unique() # Just in case I add a duplicate name by mistake

q_list <- alleg_list

# Save list to CSV
alleg_list %>%
  as_tibble(alleg_list = alleg_list) %>%
  set_names("alleg_list") %>%
  write_csv("../data/alleg_list.csv")

## Pull the TV News Archive / GDELT data for everyone in the list

In [3]:
newsflash_loop_pull <- function (q_list) {    
    nf_list <- list()
    
    # Loop over list elements
    for (i in 1:length(q_list)){
        message(paste0("Query ", i, ": ", q_list[i])) 
        pull <- query_tv(q_list[i])
        nf_list[[i]] <- pull            
        Sys.sleep(1)
    }
    
    # Remove null (i.e., zero-result) responses
    nf_list[sapply(nf_list, negate(is.null))]
}

In [4]:
list_data_pull <- newsflash_loop_pull(q_list)

Query 1: harvey weinstein
Query 2: chris savino
No results found
Query 3: robert scoble
Query 4: lockhart steele
No results found
Query 5: josh besh
Query 6: terry richardson
Query 7: leon wieseltier
Query 8: knight landesman
No results found
Query 9: rick najera
No results found
Query 10: mark halperin
Query 11: ken baker
Query 12: kevin spacey
Query 13: hamilton fish
No results found
Query 14: michael oreskes
Query 15: andy dick
Query 16: kirt webster
No results found
Query 17: brett ratner
Query 18: jeff hoover
No results found
Query 19: david guillod
No results found
Query 20: benjamin genocchio
No results found
Query 21: louis ck
Query 22: roy moore
Query 23: al franken
Query 24: richard dreyfuss
Query 25: gary goddard
Query 26: dustin hoffman
Query 27: robert knepper
No results found
Query 28: andrew kreisberg
No results found
Query 29: jeremy piven
Query 30: steven seagal
Query 31: tom sizemore
Query 32: jeffrey tambor
Query 33: george takei
Query 34: james toback
Query 35: matt

### Here's what the structure of each response looks like:

In [5]:
str(list_data_pull[1])

List of 1
 $ :List of 4
  ..$ query_details    :Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	1 obs. of  9 variables:
  .. ..$ keyword_primary    : chr "harvey weinstein"
  .. ..$ keywords_context   : chr ""
  .. ..$ stations           : chr "NATIONAL"
  .. ..$ date_start         : chr "20090702T000000Z"
  .. ..$ date_end           : chr "20171209T235959Z"
  .. ..$ timeline_resolution: chr "daily"
  .. ..$ combine_separate   : chr "SEPARATE"
  .. ..$ query_run          : chr "20171209T171535Z"
  .. ..$ total_results      : chr "4356"
  ..$ timeline         :Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	18492 obs. of  5 variables:
  .. ..$ date_start     : POSIXct[1:18492], format: "2009-07-02" "2009-07-02" ...
  .. ..$ date_end       : POSIXct[1:18492], format: "2009-07-02 23:59:59" "2009-07-02 23:59:59" ...
  .. ..$ date_resolution: chr [1:18492] "day" "day" "day" "day" ...
  .. ..$ station        : chr [1:18492] "Bloomberg" "CNBC" "CNN" "FOX Business" ...
  .. ..$ value          : int [1:184

### Extract only the name and `timeline`, and then combine all the results into one table

In [6]:
alleg_tv_data_full <- list_data_pull %>% lapply(function (data) {
    data$timeline %>%
    mutate(name=data$query_details$keyword_primary)
}) %>% bind_rows

alleg_tv_data_full %>% arrange(-value) %>% head

date_start,date_end,date_resolution,station,value,name
2017-11-16,2017-11-16 23:59:59,day,MSNBC,522,roy moore
2017-11-22,2017-11-22 23:59:59,day,CNN,462,roy moore
2017-11-15,2017-11-15 23:59:59,day,MSNBC,395,roy moore
2017-11-15,2017-11-15 23:59:59,day,CNN,385,roy moore
2017-11-14,2017-11-14 23:59:59,day,MSNBC,352,roy moore
2017-11-10,2017-11-10 23:59:59,day,MSNBC,336,roy moore


### Select only results since Oct. 1, and save them as a CSV

In [7]:
alleg_tv_data_full %>%
    mutate(date_start=as.Date(date_start)) %>% 
    filter(date_start >= as.Date("2017-10-1") & date_start < lubridate::today() - 1) %>%
    write_csv("../data/tv_daily/alleg_tv_data_recent.csv")

## Collect and aggregate hourly TV data

In [8]:
# TV explorer API only gives hourly data resolution for queries of a week or less
# Break time periods into separate, weekly queries

break_weeks <- function(start, end){
    td <- as.numeric(difftime(end, start, units = "weeks"))
        frac_wks <- td - trunc(td)

    if (trunc(td) == 0) {
        time_tbl <-
            tibble(s = start,
                   e = end)
    } else {
        time_periods_s <- 
            c(start + weeks(0:trunc(td)))

        time_periods_e <- 
            c(start + weeks(1:trunc(td))-1,
            start + weeks(trunc(td)) + round(frac_wks * 7))

        time_tbl <-
        tibble(s = time_periods_s,
              e = time_periods_e)
    }
    
    time_tbl
}


newsflash_loop_pull <- function(q_list,
                                start = ymd("2017-10-01"), 
                                end = lubridate::today() - 1,
                               timespan = "custom"){
    
    # Break full time period into week increments
    time_periods <- break_weeks(start, end)

    nf_list <- list()
      
    # Loop over each week per name
    
    for (i in 1:length(q_list)){
        for (t in 1:dim(time_periods)[1]){
            message(paste0("Query ", i, " (", q_list[i], ", ", time_periods$s[t], ")")) 
            pull <- query_tv(q_list[i],
                             start_date = time_periods$s[t], 
                             end_date = time_periods$e[t], 
                             timespan = timespan)
            
                if (!is.null(pull)){
                    timeline <- 
                        pull$timeline %>%
                        as.tibble() %>%
                        cbind(., name = q_list[i], time_period = time_periods$s[t])
                } else {
                    timeline <- NULL                   
                    }

                nf_list[[paste(i, t, sep="_")]] <- timeline
        }
        
        Sys.sleep(1)
    }
        
     nf_list
}

In [9]:
list_data_pull <- newsflash_loop_pull(q_list)

Query 1 (harvey weinstein, 2017-10-01)
Query 1 (harvey weinstein, 2017-10-08)
Query 1 (harvey weinstein, 2017-10-15)
Query 1 (harvey weinstein, 2017-10-22)
Query 1 (harvey weinstein, 2017-10-29)
Query 1 (harvey weinstein, 2017-11-05)
Query 1 (harvey weinstein, 2017-11-12)
Query 1 (harvey weinstein, 2017-11-19)
Query 1 (harvey weinstein, 2017-11-26)
Query 1 (harvey weinstein, 2017-12-03)
Query 2 (chris savino, 2017-10-01)
No results found
Query 2 (chris savino, 2017-10-08)
No results found
Query 2 (chris savino, 2017-10-15)
No results found
Query 2 (chris savino, 2017-10-22)
No results found
Query 2 (chris savino, 2017-10-29)
No results found
Query 2 (chris savino, 2017-11-05)
No results found
Query 2 (chris savino, 2017-11-12)
No results found
Query 2 (chris savino, 2017-11-19)
No results found
Query 2 (chris savino, 2017-11-26)
No results found
Query 2 (chris savino, 2017-12-03)
No results found
Query 3 (robert scoble, 2017-10-01)
No results found
Query 3 (robert scoble, 2017-10-08)
N

No results found
Query 17 (brett ratner, 2017-10-22)
No results found
Query 17 (brett ratner, 2017-10-29)
No results found
Query 17 (brett ratner, 2017-11-05)
Query 17 (brett ratner, 2017-11-12)
No results found
Query 17 (brett ratner, 2017-11-19)
Query 17 (brett ratner, 2017-11-26)
No results found
Query 17 (brett ratner, 2017-12-03)
Query 18 (jeff hoover, 2017-10-01)
No results found
Query 18 (jeff hoover, 2017-10-08)
No results found
Query 18 (jeff hoover, 2017-10-15)
No results found
Query 18 (jeff hoover, 2017-10-22)
No results found
Query 18 (jeff hoover, 2017-10-29)
No results found
Query 18 (jeff hoover, 2017-11-05)
No results found
Query 18 (jeff hoover, 2017-11-12)
No results found
Query 18 (jeff hoover, 2017-11-19)
No results found
Query 18 (jeff hoover, 2017-11-26)
No results found
Query 18 (jeff hoover, 2017-12-03)
No results found
Query 19 (david guillod, 2017-10-01)
No results found
Query 19 (david guillod, 2017-10-08)
No results found
Query 19 (david guillod, 2017-10-15

No results found
Query 34 (james toback, 2017-10-08)
No results found
Query 34 (james toback, 2017-10-15)
No results found
Query 34 (james toback, 2017-10-22)
Query 34 (james toback, 2017-10-29)
Query 34 (james toback, 2017-11-05)
No results found
Query 34 (james toback, 2017-11-12)
No results found
Query 34 (james toback, 2017-11-19)
Query 34 (james toback, 2017-11-26)
No results found
Query 34 (james toback, 2017-12-03)
No results found
Query 35 (matthew weiner, 2017-10-01)
No results found
Query 35 (matthew weiner, 2017-10-08)
No results found
Query 35 (matthew weiner, 2017-10-15)
No results found
Query 35 (matthew weiner, 2017-10-22)
No results found
Query 35 (matthew weiner, 2017-10-29)
No results found
Query 35 (matthew weiner, 2017-11-05)
No results found
Query 35 (matthew weiner, 2017-11-12)
Query 35 (matthew weiner, 2017-11-19)
No results found
Query 35 (matthew weiner, 2017-11-26)
No results found
Query 35 (matthew weiner, 2017-12-03)
No results found
Query 36 (stephen blackw

No results found
Query 50 (garrison keillor, 2017-11-05)
No results found
Query 50 (garrison keillor, 2017-11-12)
No results found
Query 50 (garrison keillor, 2017-11-19)
No results found
Query 50 (garrison keillor, 2017-11-26)
Query 50 (garrison keillor, 2017-12-03)
Query 51 (john conyers, 2017-10-01)
No results found
Query 51 (john conyers, 2017-10-08)
Query 51 (john conyers, 2017-10-15)
No results found
Query 51 (john conyers, 2017-10-22)
Query 51 (john conyers, 2017-10-29)
No results found
Query 51 (john conyers, 2017-11-05)
No results found
Query 51 (john conyers, 2017-11-12)
Query 51 (john conyers, 2017-11-19)
Query 51 (john conyers, 2017-11-26)
Query 51 (john conyers, 2017-12-03)
Query 52 (israel horovitz, 2017-10-01)
No results found
Query 52 (israel horovitz, 2017-10-08)
No results found
Query 52 (israel horovitz, 2017-10-15)
No results found
Query 52 (israel horovitz, 2017-10-22)
No results found
Query 52 (israel horovitz, 2017-10-29)
No results found
Query 52 (israel horovit

Note that here I'm pulling data a week at a time. The API returns null results where there are no mentions for a given name in a given week. This means there's a lot of implied missing data in the combined table but the data are all deterministically missing (ie, not random or errors) and can be assumed to be zero.

### Response structure

In [10]:
# A look at the structure of what I get back - just the timeline data here.
str(list_data_pull[1:4])

List of 4
 $ 1_1:'data.frame':	1008 obs. of  7 variables:
  ..$ date_start     : POSIXct[1:1008], format: "2017-10-01 00:00:00" "2017-10-01 00:00:00" ...
  ..$ date_end       : POSIXct[1:1008], format: "2017-10-01 00:29:59" "2017-10-01 00:29:59" ...
  ..$ date_resolution: chr [1:1008] "30min" "30min" "30min" "30min" ...
  ..$ station        : chr [1:1008] "Bloomberg" "CNBC" "CNN" "FOX Business" ...
  ..$ value          : int [1:1008] 0 0 0 0 0 0 0 0 0 0 ...
  ..$ name           : Factor w/ 1 level "harvey weinstein": 1 1 1 1 1 1 1 1 1 1 ...
  ..$ time_period    : Date[1:1008], format: "2017-10-01" "2017-10-01" ...
 $ 1_2:'data.frame':	1008 obs. of  7 variables:
  ..$ date_start     : POSIXct[1:1008], format: "2017-10-08 00:00:00" "2017-10-08 00:00:00" ...
  ..$ date_end       : POSIXct[1:1008], format: "2017-10-08 00:29:59" "2017-10-08 00:29:59" ...
  ..$ date_resolution: chr [1:1008] "30min" "30min" "30min" "30min" ...
  ..$ station        : chr [1:1008] "Bloomberg" "CNBC" "CNN" "FOX 

### Combine into one table

In [11]:
show_mentions_recent_data <- list_data_pull %>% 
    lapply(function (x) { x %>% mutate(name=as.character(name)) }) %>%
    bind_rows

### Combined table size

In [12]:
dim(show_mentions_recent_data)

[1] 64152     7

### Write to CSV

In [13]:
write_csv(show_mentions_recent_data, "../data/tv_hourly/show_mentions_recent_data.csv")

## Collect Google Trends data

For google trends, I need all queries to be on the same scale. The index score returned is not an absolute measure; it's measured relative to the max value of a given query in the period searched. So each name is queried together with the name that generates the absolute maximum volume of all names queried.

In [14]:
leader_name <- "matt lauer"

unleader <- function(x) x != leader_name

q_list <-
  alleg_list %>%
  as.list() %>%
map_if(., unleader, c, leader_name)  


In [15]:
gtrends_loop_pull <- function(q_list, time, geo=c("US"), s=1){
      gtrends_list <- list()
      
    # Loop over list elements    
    for (i in 1:length(q_list)){
        message(paste0("Query ", i, ": ", q_list[i]))
        pull <- gtrends(q_list[[i]], geo = geo, time = time)
        gtrends_list[[i]] <- pull
        Sys.sleep(s)
        }
     gtrends_list
}

In [16]:
time_of_interest <- paste0("2017-10-01 ",as.character(lubridate::today()-1))

gtrends_pull <- gtrends_loop_pull(q_list = q_list,
                                  geo = c("US"), 
                                  time = time_of_interest)

Query 1: c("harvey weinstein", "matt lauer")
Query 2: c("chris savino", "matt lauer")
Query 3: c("robert scoble", "matt lauer")
Query 4: c("lockhart steele", "matt lauer")
Query 5: c("josh besh", "matt lauer")
Query 6: c("terry richardson", "matt lauer")
Query 7: c("leon wieseltier", "matt lauer")
Query 8: c("knight landesman", "matt lauer")
Query 9: c("rick najera", "matt lauer")
Query 10: c("mark halperin", "matt lauer")
Query 11: c("ken baker", "matt lauer")
Query 12: c("kevin spacey", "matt lauer")
Query 13: c("hamilton fish", "matt lauer")
Query 14: c("michael oreskes", "matt lauer")
Query 15: c("andy dick", "matt lauer")
Query 16: c("kirt webster", "matt lauer")
Query 17: c("brett ratner", "matt lauer")
Query 18: c("jeff hoover", "matt lauer")
Query 19: c("david guillod", "matt lauer")
Query 20: c("benjamin genocchio", "matt lauer")
Query 21: c("louis ck", "matt lauer")
Query 22: c("roy moore", "matt lauer")
Query 23: c("al franken", "matt lauer")
Query 24: c("richard dreyfuss", 

### Combine list of responses into one table

In [17]:
alleg_search_long <-
map2(gtrends_pull, alleg_list, function(gp, al){
    gp$interest_over_time %>%
    spread(keyword, hits) %>%
    select(date, !!al) %>%
    gather(!!al, value, -date) %>%
    set_names(c("date","name","value"))
}) %>% bind_rows()

alleg_search_long %>% head
alleg_search_long %>% tail

date,name,value
2017-10-01,harvey weinstein,0
2017-10-02,harvey weinstein,0
2017-10-03,harvey weinstein,0
2017-10-04,harvey weinstein,0
2017-10-05,harvey weinstein,2
2017-10-06,harvey weinstein,5


,date,name,value
4415,2017-12-02,warren moon,0
4416,2017-12-03,warren moon,0
4417,2017-12-04,warren moon,0
4418,2017-12-05,warren moon,0
4419,2017-12-06,warren moon,0
4420,2017-12-07,warren moon,1


In [18]:
# Check the max values of search volume for each name:

# We should see exactly one person has a score of 100 and everyone else has a lower score
# That person should be `leader_name`
# Otherwise it's very likely something's wrong with the data collection

gtrends_max_values <-
alleg_search_long %>%
  group_by(name) %>%
  summarise(max_value_by_name = max(value, na.rm=TRUE)) %>%
  arrange(-max_value_by_name)

gtrends_max_values

name,max_value_by_name
matt lauer,100
kevin spacey,29
harvey weinstein,20
charlie rose,19
louis ck,16
al franken,12
garrison keillor,6
roy moore,6
russell simmons,4
brett ratner,3


In [19]:
# Drop names from long table whose max value is zero
# (Google data is truncated so zero values indicate a value less than one, but we don't know how much)

alleg_search_long <-
alleg_search_long %>%
  filter(name %in% 
         (gtrends_max_values %>% filter(max_value_by_name > 0) %>% .$name)
         )

## Save as CSV

In [20]:
## Google trends data
# Stash each of the datasets to generate visualizations

write_csv(alleg_search_long, "../data/google_trends/alleg_search_long.csv")

write_csv(gtrends_max_values, "../data/google_trends/gtrends_max_values.csv")

---

---

---